# Torchserve on SageMaker

# Create Inference script

In [25]:
! pygmentize 2_src/pipeline_predictor.py

import torch
import os
import json
from transformers import pipeline


JSON_CONTENT_TYPE = "application/json"


def model_fn(model_dir):
    model_pipeline = pipeline(
        "question-answering",
        model=os.path.join(model_dir, "distilbert-base-uncased-distilled-squad"),
    )
    return model_pipeline


def input_fn(serialized_input_data, content_type=JSON_CONTENT_TYPE):
    print(content_type)
    if content_type == JSON_CONTENT_TYPE:
        input_data = json.loads(serialized_input_data)
        return input_data
    else:
        Exception("Requested unsupported ContentType in Accept: " + content_type)


def predict_fn(input_data, model_pipeline):
    try:
        results = model_pipeline(
            question=input_data["question"], context=input_data["context"]
        )
    except Exception as e:
        print(e)
        return str(e)
    return results


def output_fn(prediction_output, accept=JSON_CONTENT_TYPE):
    if accept == JSON_CONTENT_TYPE:
        return json.d

## Model Packaging

In [26]:
# Download artifacts for Q&A model

! mkdir distilbert-base-uncased-distilled-squad
! wget https://huggingface.co/distilbert-base-uncased-distilled-squad/resolve/main/pytorch_model.bin -P distilbert-base-uncased-distilled-squad
! wget https://huggingface.co/distilbert-base-uncased-distilled-squad/resolve/main/tokenizer.json -P distilbert-base-uncased-distilled-squad
! wget https://huggingface.co/distilbert-base-uncased-distilled-squad/resolve/main/tokenizer_config.json -P distilbert-base-uncased-distilled-squad
! wget https://huggingface.co/distilbert-base-uncased-distilled-squad/resolve/main/vocab.txt -P distilbert-base-uncased-distilled-squad
! wget https://huggingface.co/distilbert-base-uncased-distilled-squad/resolve/main/config.json -P distilbert-base-uncased-distilled-squad

zu7r4nfrsk-algo-1-9oixb | 2022-08-22 01:17:08,458 [INFO ] pool-2-thread-2 TS_METRICS - CPUUtilization.Percent:0.0|#Level:Host|#hostname:6a149ec7ff42,timestamp:1661131028
zu7r4nfrsk-algo-1-9oixb | 2022-08-22 01:17:08,459 [INFO ] pool-2-thread-2 TS_METRICS - DiskAvailable.Gigabytes:162.69421005249023|#Level:Host|#hostname:6a149ec7ff42,timestamp:1661131028
zu7r4nfrsk-algo-1-9oixb | 2022-08-22 01:17:08,459 [INFO ] pool-2-thread-2 TS_METRICS - DiskUsage.Gigabytes:121.06816101074219|#Level:Host|#hostname:6a149ec7ff42,timestamp:1661131028
zu7r4nfrsk-algo-1-9oixb | 2022-08-22 01:17:08,459 [INFO ] pool-2-thread-2 TS_METRICS - DiskUtilization.Percent:42.7|#Level:Host|#hostname:6a149ec7ff42,timestamp:1661131028
zu7r4nfrsk-algo-1-9oixb | 2022-08-22 01:17:08,459 [INFO ] pool-2-thread-2 TS_METRICS - MemoryAvailable.Megabytes:2416.23828125|#Level:Host|#hostname:6a149ec7ff42,timestamp:1661131028
zu7r4nfrsk-algo-1-9oixb | 2022-08-22 01:17:08,459 [INFO ] pool-2-thread-2 TS_METRICS - MemoryUsed.Megabytes

In [27]:
!tar -C "$PWD" -czf distilbert-base-uncased-distilled-squad.tar.gz  distilbert-base-uncased-distilled-squad/

zu7r4nfrsk-algo-1-9oixb | 2022-08-22 01:18:08,388 [INFO ] pool-2-thread-2 TS_METRICS - CPUUtilization.Percent:50.0|#Level:Host|#hostname:6a149ec7ff42,timestamp:1661131088
zu7r4nfrsk-algo-1-9oixb | 2022-08-22 01:18:08,390 [INFO ] pool-2-thread-2 TS_METRICS - DiskAvailable.Gigabytes:162.69420623779297|#Level:Host|#hostname:6a149ec7ff42,timestamp:1661131088
zu7r4nfrsk-algo-1-9oixb | 2022-08-22 01:18:08,390 [INFO ] pool-2-thread-2 TS_METRICS - DiskUsage.Gigabytes:121.06816482543945|#Level:Host|#hostname:6a149ec7ff42,timestamp:1661131088
zu7r4nfrsk-algo-1-9oixb | 2022-08-22 01:18:08,390 [INFO ] pool-2-thread-2 TS_METRICS - DiskUtilization.Percent:42.7|#Level:Host|#hostname:6a149ec7ff42,timestamp:1661131088
zu7r4nfrsk-algo-1-9oixb | 2022-08-22 01:18:08,391 [INFO ] pool-2-thread-2 TS_METRICS - MemoryAvailable.Megabytes:2417.24609375|#Level:Host|#hostname:6a149ec7ff42,timestamp:1661131088
zu7r4nfrsk-algo-1-9oixb | 2022-08-22 01:18:08,392 [INFO ] pool-2-thread-2 TS_METRICS - MemoryUsed.Megabyte

In [1]:
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()
role = get_execution_role()

bucket = sagemaker_session.default_bucket()
prefix = 'torchserve'
s3_path = 's3://{}/{}'.format(bucket, prefix)


model_data = sagemaker_session.upload_data('distilbert-base-uncased-distilled-squad.tar.gz',
                                           bucket,
                                           os.path.join(prefix, 'model-artifacts'))

In [2]:
print(model_data)

s3://sagemaker-us-east-1-941656036254/torchserve/model-artifacts/distilbert-base-uncased-distilled-squad.tar.gz


## Remote Endpoint

In [9]:
from sagemaker.pytorch import PyTorchModel

env = {
    "SAGEMAKER_TS_BATCH_SIZE": "3",
    "SAGEMAKER_TS_MAX_BATCH_DELAY": "1000",
    "SAGEMAKER_TS_RESPONSE_TIMEOUT" : "120",
    "SAGEMAKER_TS_MIN_WORKERS" : "1",
    "SAGEMAKER_TS_MAX_WORKERS" : "2"
    }


# Note: You can update the 'torchserve-predictor.py' file as needed according to the model you want to use (ie BERT) 
model = PyTorchModel(model_data=model_data,
                   role=role, 
                   entry_point='pipeline_predictor.py',
                   source_dir='2_src',
                   framework_version='1.9.0',
                   py_version='py38',
                   env=env,
                   sagemaker_session=sagemaker_session)

In [10]:
import json

context = r"""
The Amazon rainforest (Portuguese: Floresta Amazônica or Amazônia; Spanish: Selva Amazónica, Amazonía or usually Amazonia; French: Forêt amazonienne; Dutch: Amazoneregenwoud), also known in English as Amazonia or the Amazon Jungle, is a moist broadleaf forest that covers most of the Amazon basin of South America. This basin encompasses 7,000,000 square kilometres (2,700,000 sq mi), of which 5,500,000 square kilometres (2,100,000 sq mi) are covered by the rainforest. This region includes territory belonging to nine nations. The majority of the forest is contained within Brazil, with 60% of the rainforest, followed by Peru with 13%, Colombia with 10%, and with minor amounts in Venezuela, Ecuador, Bolivia, Guyana, Suriname and French Guiana. States or departments in four nations contain "Amazonas" in their names. The Amazon represents over half of the planet's remaining rainforests, and comprises the largest and most biodiverse tract of tropical rainforest in the world, with an estimated 390 billion individual trees divided into 16,000 species.
"""

question="What kind of forest is Amazon?"


data = {"question":question, "context":context}



In [12]:
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer


remote_predictor = model.deploy(initial_instance_count=1, instance_type="ml.g4dn.4xlarge", serializer=JSONSerializer(), deserializer=JSONDeserializer())

-------------!

In [14]:
remote_predictor.predict(data)

In [8]:
remote_predictor.delete_endpoint()